In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
import matplotlib.pyplot as plt

import hog

Using TensorFlow backend.


# Template for Keras tasks of sheet 5

In [5]:
# Load MNIST data
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
img_rows, img_cols = 28, 28

# Normalize it
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# Store it in the correct format for Keras
# The image data has a single channel (grayscale values)
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
# Store the labels in the correct format for Keras
Y_train = keras.utils.np_utils.to_categorical(y_train, 10)
Y_test = keras.utils.np_utils.to_categorical(y_test, 10)

To use `X_train` for fully-connected inputs, reshape it. Use the `input_shape` variable in the first layer of your networks.

In [ ]:
def plot_history(history):
    """Create a plot showing the training history of `model.fit`.
    
    Example:
        history = model.fit(...)
        plot_history(history)
    """
    x = range(history.params['epochs'])
    acc, val_acc = history.history['acc'], history.history.get('val_acc')
    f, axarr = plt.subplots(2, sharex=True)
    axarr[0].set_title('accuracy')
    axarr[0].plot(x, acc, label='train')
    if val_acc:
        axarr[0].plot(x, val_acc, label='validation')
    axarr[0].legend()
    
    loss, val_loss = history.history['loss'], history.history.get('val_loss')
    axarr[1].set_title('loss')
    axarr[1].plot(x, loss, label='train')
    if val_loss:
        axarr[1].plot(x, val_loss, label='validation')
    axarr[1].legend()

In [13]:
model1 = keras.models.Sequential()
model1.add(Dense(units=128, activation='relu', input_dim=28*28))
model1.add(Dropout(0.3))
model1.add(Dense(units=128, activation='relu', input_dim=128))
model1.add(Dropout(0.3))
model1.add(Dense(units=10, activation='softmax'))
model1.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])


history = model1.fit(X_train.reshape(60000,-1), Y_train.reshape(60000,-1), epochs=50, batch_size=128)
loss_and_metrics = model1.evaluate(X_test.reshape(10000,-1), Y_test.reshape(10000,-1), batch_size=128)


print(loss_and_metrics)

Epoch 1/50
60000/60000 [==============================] - 3s 56us/step - loss: 1.5817 - acc: 0.5039
Epoch 2/50
60000/60000 [==============================] - 3s 54us/step - loss: 0.8043 - acc: 0.7530
Epoch 3/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.6196 - acc: 0.8124
Epoch 4/50
60000/60000 [==============================] - 4s 59us/step - loss: 0.5352 - acc: 0.8386
Epoch 5/50
60000/60000 [==============================] - 3s 54us/step - loss: 0.4836 - acc: 0.8559
Epoch 6/50
60000/60000 [==============================] - 3s 55us/step - loss: 0.4450 - acc: 0.8673
Epoch 7/50
60000/60000 [==============================] - 3s 57us/step - loss: 0.4171 - acc: 0.8755
Epoch 8/50
60000/60000 [==============================] - 3s 54us/step - loss: 0.3937 - acc: 0.8821
Epoch 9/50
60000/60000 [==============================] - 3s 54us/step - loss: 0.3751 - acc: 0.8878
Epoch 10/50
60000/60000 [==============================] - 3s 54us/step - loss: 0.3569 - acc: 0.8939

KeyboardInterrupt: 

## Pedestrian Bonus Task

In [2]:
from sklearn.svm import LinearSVC
import scipy.io
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [3]:
ped_data = scipy.io.loadmat("data/pca_ped_25x50.mat")
ped_images = ped_data['ped_train_int_25x50'][:,1:]
garb_images = ped_data['garb_train_int_25x50'][:,1:]
ped_test_images = ped_data['ped_test_int_25x50'][:,1:]
garb_test_images = ped_data['garb_test_int_25x50'][:,1:]
del ped_data


min_max = MinMaxScaler()
min_max.fit(np.vstack((ped_images, garb_images)))

ped_images = min_max.transform(ped_images)
garb_images = min_max.transform(garb_images)
y_train = np.concatenate((np.ones(len(ped_images)), np.zeros(len(garb_images))))

ped_test_images = min_max.transform(ped_test_images)
garb_test_images = min_max.transform(garb_test_images)
y_test = np.concatenate((np.ones(len(ped_test_images)), np.zeros(len(garb_test_images))))

In [5]:
img_rows, img_cols = 25, 50
X_train = np.vstack((ped_images, garb_images)).astype('float32')
X_test = np.vstack((ped_test_images, garb_test_images)).astype('float32')
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
Y_train = keras.utils.np_utils.to_categorical(y_train,2)
Y_test = keras.utils.np_utils.to_categorical(y_test, 2)

print(X_test.shape)

(1000, 25, 50, 1)


In [ ]:
from dual_path_network import DPN92, DPN98, DPN107, DPN137
import cv2

X_resized_train = np.zeros((3000, 25*5, 50*5, 1))
for i in range(X_train.shape[0]):
    X_resized_train[i] = cv2.resize(X_train[i], dsize=(50*5, 25*5), interpolation=cv2.INTER_CUBIC).resize(125,250,1)

X_resized_test = np.zeros((1000, 25*5, 50*5, 1))
for i in range(X_test.shape[0]):
    X_resized_test[i] = cv2.resize(X_test[i], dsize=(50*5, 25*5), interpolation=cv2.INTER_CUBIC).resize(125,250,1)

       
model = DPN92(input_shape=(125, 250, 1)) 

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(X_resized_train, Y_train, batch_size=32, epochs=10)
score = model.evaluate(X_resized_test, Y_test, batch_size=32)
print(score)

BLOCK 2
BLOCK 3
BLOCK 4
Epoch 1/10


In [32]:
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(25, 50, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))



sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=32, epochs=10)
score = model.evaluate(X_test, Y_test, batch_size=32)
print(score)

Epoch 1/10
3000/3000 [==============================] - 16s 5ms/step - loss: 0.6911 - acc: 0.5123
Epoch 2/10
3000/3000 [==============================] - 15s 5ms/step - loss: 0.6312 - acc: 0.6410
Epoch 3/10
3000/3000 [==============================] - 16s 5ms/step - loss: 0.5646 - acc: 0.7140
Epoch 4/10
3000/3000 [==============================] - 17s 6ms/step - loss: 0.4824 - acc: 0.7740
Epoch 5/10
3000/3000 [==============================] - 16s 5ms/step - loss: 0.4088 - acc: 0.8213
Epoch 6/10
3000/3000 [==============================] - 16s 5ms/step - loss: 0.3579 - acc: 0.8473
Epoch 7/10
3000/3000 [==============================] - 16s 5ms/step - loss: 0.3053 - acc: 0.8727
Epoch 8/10
3000/3000 [==============================] - 18s 6ms/step - loss: 0.2739 - acc: 0.8867
Epoch 9/10
3000/3000 [==============================] - 17s 6ms/step - loss: 0.2416 - acc: 0.9037
Epoch 10/10
1000/1000 [==============================] - 3s 3ms/step
[0.392188932955265, 0.841]


In [33]:
acc = model.evaluate(X_test, Y_test, batch_size=32)
print(acc)

1000/1000 [==============================] - 2s 2ms/step
[0.392188932955265, 0.841]
